<a href="https://colab.research.google.com/github/nedokormysh/GB_PyTorch/blob/lesson4/GB_PyTorch_hw_4_dirty.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Домашнее задание**

Обучите CNN (самописная) на CIFAR-100.

Обучите CNN на CIFAR-100 через дообучение ImageNet Resnet-50.

*Обучите CNN на CIFAR-100 через дообучение ImageNet Resnet-50 с аугментацией данных.

## Обучите CNN (самописная) на CIFAR-100.

In [1]:
import numpy as np
import torch

from torch import nn
from torch.nn import functional as F
from PIL import Image
from torchvision import transforms, datasets
from tqdm import tqdm

from sklearn.model_selection import train_test_split

Проверка на доступное устройство.

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

Загрузка датасета. Создание класса.

In [3]:
dataset = datasets.CIFAR100(root='data/', train=True, download=True)


class MyOwnCifar(torch.utils.data.Dataset):
   
    def __init__(self, init_dataset, transform=None):
        self._base_dataset = init_dataset
        self.transform = transform

    def __len__(self):
        return len(self._base_dataset)

    def __getitem__(self, idx):
        img = self._base_dataset[idx][0]
        if self.transform is not None:
            img = self.transform(img)
        return img, self._base_dataset[idx][1]

  0%|          | 0/169001437 [00:00<?, ?it/s]

Extracting data/cifar-100-python.tar.gz to data/


In [4]:
trans_actions = transforms.Compose([
                                    # transforms.Resize(44),
                                    transforms.RandomInvert(p=0.6),
                                    transforms.RandomGrayscale(0.3),
                                    transforms.RandomRotation(degrees=(0,5)),
                                    # # transforms.RandomCrop(32, padding=4),
                                    transforms.ToTensor()
                                    ])


def train_valid_split(Xt):
    X_train, X_test = train_test_split(Xt, test_size=0.3, random_state=42)
    return X_train, X_test

train_dataset, valid_dataset = train_valid_split(dataset)

train_dataset = MyOwnCifar(train_dataset, trans_actions)
valid_dataset = MyOwnCifar(valid_dataset, transforms.ToTensor())

Загрузчики

In [5]:
train_loader = torch.utils.data.DataLoader(train_dataset,
                          batch_size=128,
                          shuffle=True,
                          num_workers=2)
valid_loader = torch.utils.data.DataLoader(valid_dataset,
                          batch_size=128,
                          shuffle=False,
                          num_workers=1)

Сеть

In [6]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.dp_one = nn.Dropout(0.2)
        self.dp_two = nn.Dropout(0.2)
        
        self.bn_one = torch.nn.BatchNorm2d(3) 
        self.conv_one = torch.nn.Conv2d(3, 45, 3, padding_mode='zeros', padding=1) #input channels, output channels, kernel_size
        # self.conv_one = torch.nn.Conv2d(3, 45, 3)
        self.bn_two = torch.nn.BatchNorm2d(45) 
        self.conv_two = torch.nn.Conv2d(45, 90, 3)
        self.bn_three = torch.nn.BatchNorm2d(90)
        self.conv_three = torch.nn.Conv2d(90, 180, 3)
        self.bn_four = torch.nn.BatchNorm2d(180)
        self.fc1 = torch.nn.Linear(720, 360)
        self.fc2 = torch.nn.Linear(360, 180)
        self.out = torch.nn.Linear(180, 100)
        
    def forward(self, x):
        x = self.bn_one(x) # 3 канала 32 на 32 (32*3*2=192x32)
        x = self.conv_one(x) # 45 каналов 32 на 32 (192*15 = 2880x32) Hout = ((H_in + 2*padding(=0) - dilation(=1)*(kernel_size-1) - 1)/stride(=1)) + 1
        # = ((32 + 0 - (3-1) - 1)/1) + 1 = 29 + 1 = 30
        # Это если без паддингов (padding=0). с паддингами по-другому.
        x = F.relu(x) 
        x = F.max_pool2d(x, 2) # 45 
        
        x = self.bn_two(x) # 45
        x = self.conv_two(x) # 90
        x = F.relu(x)
        x = F.max_pool2d(x, 2) # 90 
        
        x = self.bn_three(x) # 90 
        x = self.conv_three(x) # 180 
        x = F.leaky_relu(x, 0.1)
        x = F.max_pool2d(x, 2) # 180 
        
        x = self.bn_four(x) # 180 
        x = x.view(x.size(0), -1) # 1 вектор 180
        x = self.dp_one(x)
        x = self.fc1(x)
        x = F.relu(x)        
        x = self.fc2(x)
        x = F.relu(x)
        return self.out(x)
       
net = Net().to(device)
print(net)

Net(
  (dp_one): Dropout(p=0.2, inplace=False)
  (dp_two): Dropout(p=0.2, inplace=False)
  (bn_one): BatchNorm2d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv_one): Conv2d(3, 45, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn_two): BatchNorm2d(45, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv_two): Conv2d(45, 90, kernel_size=(3, 3), stride=(1, 1))
  (bn_three): BatchNorm2d(90, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv_three): Conv2d(90, 180, kernel_size=(3, 3), stride=(1, 1))
  (bn_four): BatchNorm2d(180, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc1): Linear(in_features=720, out_features=360, bias=True)
  (fc2): Linear(in_features=360, out_features=180, bias=True)
  (out): Linear(in_features=180, out_features=100, bias=True)
)


По размерностям

https://discuss.pytorch.org/t/how-do-i-get-the-correct-tensor-dimensions-after-conv2d-layer-keras-to-pytorch/131925/3

https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html

https://discuss.pytorch.org/t/what-is-1-in-output-shape-of-a-model-in-torch-summary/67790

https://dingyan89.medium.com/calculating-parameters-of-convolutional-and-fully-connected-layers-with-keras-186590df36c6

https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html#torch.nn.Conv2d

https://datascience.stackexchange.com/questions/40906/determining-size-of-fc-layer-after-conv-layer-in-pytorch

https://iq.opengenus.org/output-size-of-convolution/#:~:text=In%20short%2C%20the%20answer%20is,%2F%20(stride%20height)%20%2B%201


https://dingyan89.medium.com/calculating-parameters-of-convolutional-and-fully-connected-layers-with-keras-186590df36c6

https://www.cs.toronto.edu/~lczhang/360/lec/w04/convnet.html#:~:text=Parameters%20of%20a%20Convolutional%20Layer&text=There%20is%20one%20bias%20for,element%20in%20that%20output%20channel.

Самая простая
https://stackoverflow.com/questions/70231487/output-dimensions-of-convolution-in-pytorch

In [7]:
from torchsummary import summary

summary(net.to(device), input_size=(3, 32, 32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
       BatchNorm2d-1            [-1, 3, 32, 32]               6
            Conv2d-2           [-1, 45, 32, 32]           1,260
       BatchNorm2d-3           [-1, 45, 16, 16]              90
            Conv2d-4           [-1, 90, 14, 14]          36,540
       BatchNorm2d-5             [-1, 90, 7, 7]             180
            Conv2d-6            [-1, 180, 5, 5]         145,980
       BatchNorm2d-7            [-1, 180, 2, 2]             360
           Dropout-8                  [-1, 720]               0
            Linear-9                  [-1, 360]         259,560
           Linear-10                  [-1, 180]          64,980
           Linear-11                  [-1, 100]          18,100
Total params: 527,056
Trainable params: 527,056
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.01
Forward/

In [8]:
optimizer = torch.optim.Adam(net.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

In [9]:
num_epochs = 5
net.train()

for epoch in range(num_epochs):  
    running_loss, running_items, running_right = 0.0, 0.0, 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)

        # обнуляем градиент
        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # выводим статистику о процессе обучения
        running_loss += loss.item()
        running_items += len(labels)
        running_right += (labels == torch.max(outputs, 1)[1]).sum()
        
        # выводим статистику о процессе обучения
        if i % 300 == 0:    # печатаем каждые 300 mini-batches
            net.eval()
            
            print(f'Epoch [{epoch + 1}/{num_epochs}]. ' \
                  f'Step [{i + 1}/{len(train_loader)}]. ' \
                  f'Loss: {running_loss / running_items:.3f}. ' \
                  f'Acc: {running_right / running_items:.3f}', end='. ')
            running_loss, running_items, running_right = 0.0, 0.0, 0.0

            test_running_right, test_running_total = 0.0, 0.0
            for i, data in enumerate(valid_loader):
            
                test_outputs = net(data[0].to(device))
                test_running_total += len(data[1])
                test_running_right += (data[1].to(device) == torch.max(test_outputs, 1)[1]).sum()
            
            print(f'Test acc: {test_running_right / test_running_total:.3f}')
        
        net.train()
        
print('Training is finished!')

Epoch [1/5]. Step [1/274]. Loss: 0.036. Acc: 0.000. Test acc: 0.013
Epoch [2/5]. Step [1/274]. Loss: 0.029. Acc: 0.102. Test acc: 0.139
Epoch [3/5]. Step [1/274]. Loss: 0.024. Acc: 0.172. Test acc: 0.236
Epoch [4/5]. Step [1/274]. Loss: 0.023. Acc: 0.320. Test acc: 0.283
Epoch [5/5]. Step [1/274]. Loss: 0.021. Acc: 0.344. Test acc: 0.304
Training is finished!


## Обучите CNN на CIFAR-100 через дообучение ImageNet Resnet-50.

Загрузим предобученную сеть

In [10]:
from torchvision import models

resnet50 = models.resnet50(pretrained=True)
print(resnet50)

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [11]:
from torchsummary import summary
summary(resnet50.to(device), input_size=(3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]           4,096
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]          16,384
      BatchNorm2d-12          [-1, 256, 56, 56]             512
           Conv2d-13          [-1, 256, 56, 56]          16,384
      BatchNorm2d-14          [-1, 256,

На выходе сети 1000 классов. Нам нужно 100. Заменим выходной линейный слой. Также отключим подсчёт градиентов на предобученной части.

In [12]:
# def set_parameter_requires_grad(model, feature_extracting):
#     if feature_extracting:
#         for param in model.parameters():
#             param.requires_grad = False

# set_parameter_requires_grad(resnet50, True)

In [13]:
for param in resnet50.parameters():
    param.requires_grad = False

resnet50.fc = nn.Linear(2048, 100)
summary(resnet50.to(device), input_size=(3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]           4,096
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]          16,384
      BatchNorm2d-12          [-1, 256, 56, 56]             512
           Conv2d-13          [-1, 256, 56, 56]          16,384
      BatchNorm2d-14          [-1, 256,

In [14]:
resnet50 = resnet50.to(device)

In [15]:
train_actions = transforms.Compose([transforms.Resize(256),
                                    transforms.RandomCrop(224, padding=4), 
                                    transforms.ToTensor(),
                                    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                         std=[0.229, 0.224, 0.225])])
valid_transforms = transforms.Compose([transforms.Resize(224),
                                       transforms.ToTensor(),
                                       transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                            std=[0.229, 0.224, 0.225])])


train_dataset, valid_dataset = train_valid_split(dataset)

train_dataset = MyOwnCifar(train_dataset, train_actions)
valid_dataset = MyOwnCifar(valid_dataset, valid_transforms)

In [16]:
train_loader = torch.utils.data.DataLoader(train_dataset,
                                           batch_size=128,
                                           shuffle=True,
                                           num_workers=2)
valid_loader = torch.utils.data.DataLoader(valid_dataset,
                                           batch_size=128,
                                           shuffle=False,
                                           num_workers=1)

In [17]:
params_to_update = []
for name, param in resnet50.named_parameters():
    if param.requires_grad == True:
        params_to_update.append(param)


optimizer = torch.optim.Adam(params_to_update, lr=0.001)
criterion = nn.CrossEntropyLoss()

In [18]:
num_epochs = 5
resnet50.train()

for epoch in range(num_epochs):  
    running_loss, running_items, running_right = 0.0, 0.0, 0.0
    for i, data in enumerate(train_loader):
        inputs, labels = data[0].to(device), data[1].to(device)

        # обнуляем градиент
        optimizer.zero_grad()

        outputs = resnet50(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # выводим статистику о процессе обучения
        running_loss += loss.item()
        running_items += len(labels)
        running_right += (labels == torch.max(outputs, 1)[1]).sum()
        
        # выводим статистику о процессе обучения
        if i % 300 == 0:    # печатаем каждые 300 mini-batches
            resnet50.eval()
            
            print(f'Epoch [{epoch + 1}/{num_epochs}]. ' \
                  f'Step [{i + 1}/{len(train_loader)}]. ' \
                  f'Loss: {running_loss / running_items:.3f}. ' \
                  f'Acc: {running_right / running_items:.3f}', end='. ')
            running_loss, running_items, running_right = 0.0, 0.0, 0.0

            test_running_right, test_running_total = 0.0, 0.0
            for i, data in enumerate(valid_loader):
            
                test_outputs = resnet50(data[0].to(device))
                test_running_total += len(data[1])
                test_running_right += (data[1].to(device) == torch.max(test_outputs, 1)[1]).sum()
            
            print(f'Test acc: {test_running_right / test_running_total:.3f}')

        resnet50.train()
        
print('Training is finished!')

Epoch [1/5]. Step [1/274]. Loss: 0.036. Acc: 0.008. Test acc: 0.007
Epoch [2/5]. Step [1/274]. Loss: 0.015. Acc: 0.500. Test acc: 0.512
Epoch [3/5]. Step [1/274]. Loss: 0.013. Acc: 0.531. Test acc: 0.552
Epoch [4/5]. Step [1/274]. Loss: 0.012. Acc: 0.586. Test acc: 0.576
Epoch [5/5]. Step [1/274]. Loss: 0.010. Acc: 0.664. Test acc: 0.575
Training is finished!


Видно, что на предобученной сети можем наблюдать сильное увеличение точности на одинаковом количестве эпох.

## *Обучите CNN на CIFAR-100 через дообучение ImageNet Resnet-50 с аугментацией данных.

In [19]:
resnet50 = models.resnet50(pretrained=True)

for param in resnet50.parameters():
    param.requires_grad = False

resnet50.fc = nn.Linear(2048, 100)
# summary(resnet50.to(device), input_size=(3, 224, 224))

resnet50 = resnet50.to(device)

In [21]:
train_actions = transforms.Compose([transforms.Resize(256),
                                    transforms.RandomInvert(p=0.6),
                                    transforms.RandomGrayscale(0.3),
                                    transforms.RandomRotation(degrees=(0,5)),
                                    transforms.RandomHorizontalFlip(p=0.5),
                                    transforms.RandomCrop(224, padding=4), 
                                    transforms.ToTensor(),
                                    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                         std=[0.229, 0.224, 0.225])])
valid_transforms = transforms.Compose([transforms.Resize(224),
                                       transforms.ToTensor(),
                                       transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                            std=[0.229, 0.224, 0.225])])


train_dataset, valid_dataset = train_valid_split(dataset)

train_dataset = MyOwnCifar(train_dataset, train_actions)
valid_dataset = MyOwnCifar(valid_dataset, valid_transforms)

In [22]:
train_loader = torch.utils.data.DataLoader(train_dataset,
                                           batch_size=128,
                                           shuffle=True,
                                           num_workers=2)
valid_loader = torch.utils.data.DataLoader(valid_dataset,
                                           batch_size=128,
                                           shuffle=False,
                                           num_workers=1)

In [23]:
params_to_update = []
for name, param in resnet50.named_parameters():
    if param.requires_grad == True:
        params_to_update.append(param)


optimizer = torch.optim.Adam(params_to_update, lr=0.001)
criterion = nn.CrossEntropyLoss()

In [24]:
num_epochs = 5
resnet50.train()

for epoch in range(num_epochs):  
    running_loss, running_items, running_right = 0.0, 0.0, 0.0
    for i, data in enumerate(train_loader):
        inputs, labels = data[0].to(device), data[1].to(device)

        # обнуляем градиент
        optimizer.zero_grad()

        outputs = resnet50(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # выводим статистику о процессе обучения
        running_loss += loss.item()
        running_items += len(labels)
        running_right += (labels == torch.max(outputs, 1)[1]).sum()
        
        # выводим статистику о процессе обучения
        if i % 300 == 0:    # печатаем каждые 300 mini-batches
            resnet50.eval()
            
            print(f'Epoch [{epoch + 1}/{num_epochs}]. ' \
                  f'Step [{i + 1}/{len(train_loader)}]. ' \
                  f'Loss: {running_loss / running_items:.3f}. ' \
                  f'Acc: {running_right / running_items:.3f}', end='. ')
            running_loss, running_items, running_right = 0.0, 0.0, 0.0

            test_running_right, test_running_total = 0.0, 0.0
            for i, data in enumerate(valid_loader):
            
                test_outputs = resnet50(data[0].to(device))
                test_running_total += len(data[1])
                test_running_right += (data[1].to(device) == torch.max(test_outputs, 1)[1]).sum()
            
            print(f'Test acc: {test_running_right / test_running_total:.3f}')

        resnet50.train()
        
print('Training is finished!')

Epoch [1/5]. Step [1/274]. Loss: 0.036. Acc: 0.016. Test acc: 0.008
Epoch [2/5]. Step [1/274]. Loss: 0.025. Acc: 0.250. Test acc: 0.367
Epoch [3/5]. Step [1/274]. Loss: 0.021. Acc: 0.305. Test acc: 0.423
Epoch [4/5]. Step [1/274]. Loss: 0.020. Acc: 0.406. Test acc: 0.459
Epoch [5/5]. Step [1/274]. Loss: 0.021. Acc: 0.305. Test acc: 0.474
Training is finished!


На аналогичном количестве эпох сеть с аугментацией показала результат чуть хуже, чем без аугментации. Впрочем, возможно из-за количества данных следовало изменить параметры, например, шаг обучения или количество эпох. Вероятно сеть с аугментацией не должна проигрывать без аугментации, но возможно она обучается дольше.